In [63]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import os

In [64]:
data_path = '../sensor_data/'
os.listdir(data_path)

['s1621503',
 's1746788',
 's1701688',
 '.DS_Store',
 's1710228',
 's1737472',
 's1721039',
 's1616573',
 's1655560_files',
 's1711507',
 's1758009',
 's1660711']

In [65]:
users = ['s1621503',
 's1746788',
 's1701688',
 's1710228',
 's1721039',
 's1616573',
 's1758009',
 's1660711']
full_paths = [data_path + usr + '/' + filename for usr in users for filename in os.listdir(data_path + usr)]
header_size = 5

In [81]:
class DataRecording:  
    def __init__(self, sensor_position, sensor_side, activity, activity_id):
        self.sensor_position = sensor_position
        self.sensor_side = sensor_side
        self.activity = activity
        self.activity_id = activity_id
    
    @classmethod
    def create_recording(cls, filepath):
        with open(filepath) as f:
            head = [next(f).rstrip().split('# ')[1].split(':')[1].strip() for x in range(header_size)]
            recording = DataRecording(head[0], head[1], head[2], head[3])
        df = pd.read_csv(filepath, header=5)
        recording.data = df
        return recording

In [67]:
all_data = []
full_paths
for path in full_paths:
    try:
        all_data.append(DataRecording.create_recording(path))
    except:
        pass
        print(path)

../sensor_data/s1721039/.DS_Store
../sensor_data/s1721039/list.txt
../sensor_data/s1721039/PDIoT Data Collection Protocol.docx
../sensor_data/s1721039/my_copy.sh


In [82]:
len(all_data)

620

In [87]:
all_data[0].data

,timestamp,seq,accel_x,accel_y,accel_z
0,1601636678127,0,-0.943359,-0.271545,0.303162
1,1601636680804,1,-0.950684,-0.270569,0.294373
2,1601636680804,2,-0.942139,-0.280090,0.302185
3,1601636680805,3,-0.960449,-0.273010,0.289734
4,1601636680806,4,-0.942871,-0.263245,0.294128
...,...,...,...,...,...
380,1601636709071,380,-0.926025,-0.271057,0.346863
381,1601636709072,381,-0.927246,-0.270081,0.345398
382,1601636709072,382,-0.929932,-0.268127,0.350281
383,1601636709073,383,-0.924561,-0.269104,0.346619


In [69]:
allowed_activites = ['Climbing stairs', 'Descending stairs', 'Walking at normal speed', 'Running', 'Standing']
right_pocket_data = [x for x in all_data if 
                        x.sensor_position == 'Front pocket (trouser)' 
                            and 
                        x.sensor_side == 'Right'
                           and
                        x.activity in allowed_activites
                       ]
len(right_pocket_data)

39

In [88]:
def get_overlapping_windows(df, win_size, overlap, extract_features):
    rows,  = df.shape
    assert(overlap < 1)
    assert(overlap >= 0)
    assert(win_size <= rows)

    increment = int(win_size * (1 - overlap))
    result = []
    for i in range(0, rows - win_size, increment):
        feature_vector = extract_features(df[i:i+win_size])
        result.append(feature_vector)
    return np.array(result)


In [94]:
def get_full_window(df):
    return df

In [71]:
def get_feature_vector(df):
    return np.array([df.mean(), df.max() - df.min(), df.var(), df.std()])

In [89]:
def process_data(data_objects):
    df = pd.DataFrame()
    for d_o in data_objects:
        processed_windows = get_overlapping_windows(d_o.data['accel_x'], 50, 0.5, get_feature_vector)
        tmp_df = pd.DataFrame(processed_windows)
        tmp_df['Activity'] = d_o.activity_id
        df = pd.concat([df, tmp_df])
    return df
        

In [124]:
def split_data_into_windows(data_objects):
    df = pd.DataFrame()
    for d_o in data_objects:
        x_processed_windows = get_overlapping_windows(d_o.data['accel_x'], 50, 0.5, get_full_window)
        y_processed_windows = get_overlapping_windows(d_o.data['accel_y'], 50, 0.5, get_full_window)
        z_processed_windows = get_overlapping_windows(d_o.data['accel_z'], 50, 0.5, get_full_window)
        joined_data = np.concatenate((x_processed_windows, y_processed_windows, z_processed_windows), axis=1)
        tmp_df = pd.DataFrame(joined_data)
        tmp_df['Activity'] = d_o.activity_id
        df = pd.concat([df, tmp_df])
    return df

In [126]:
cnn_dataset = split_data_into_windows(right_pocket_data)
cnn_dataset.shape

(646, 151)

In [73]:
full_df = process_data(right_pocket_data)
full_df.shape

(646, 5)

In [74]:
from sklearn import svm

In [75]:
X = full_df.iloc[:, :4]
X.shape

(646, 4)

In [77]:
y = np.array(full_df.iloc[:, 4:]).reshape((646,))
y.shape

(646,)

In [78]:
clf = svm.SVC()
clf.fit(X, y)

SVC()

In [79]:
y_preds = clf.predict(X)

In [100]:
accuracy = (np.sum(y_preds == y)*100)/y_preds.shape[0]
print(accuracy)

64.08668730650155


In [117]:
a = np.array([[1,1,1], [2,2,2], [3,3,3]])
b = np.array([[4,4,4], [5,5,5], [6,6,6]])
c = np.array([[7,7,7], [8,8,8], [9,9,9]])
np.concatenate((a, b, c), axis=1)

array([[1, 1, 1, 4, 4, 4, 7, 7, 7],
       [2, 2, 2, 5, 5, 5, 8, 8, 8],
       [3, 3, 3, 6, 6, 6, 9, 9, 9]])